# List comprehensions: Basic syntax

- Minimal syntax: `[expression for element in iterable]`
- The square brackets are necessary—without it's a generator expression!

Let's consider a `for` loop that prints out the square root of 0 - 4:

In [1]:
from math import sqrt

for i in range(5):
    print(sqrt(i))


0.0
1.0
1.4142135623730951
1.7320508075688772
2.0


Now let's implement this with a `list` comprehension:

In [3]:
_ = [print(sqrt(i)) for i in range(5)]

0.0
1.0
1.4142135623730951
1.7320508075688772
2.0


#### Filtering list comprehensions

- Filtering syntax: `[expression for element in iterable if expression]`
- This is an alternative (in many cases) to the `continue` statement

Let's consider a `for` loop that skips all odd numbers:

In [4]:
for i in range(5):
    if i%2:
        continue
    print(i)

0
2
4


Now let's implement this with a `list` comprehension:

In [5]:
_ = [print(i) for i in range(5) if not i%2]

0
2
4


#### Breaking list comprehensions

- There is no way to `break` a list comprehension
- Although you can do this with a generator expression, which we will meet later this section!

Let's consider a `for` loop that iteratres over an infinite generator function, `fibonacci()`, until a number that is larger than 10 is encountered:

In [6]:
def fibonacci():
    
    yield 1
    yield 1
    l = [1, 1]
    while True:
        l = [l[-1], sum(l[-2:])]
        yield l[-1]

        
for i in fibonacci():
    if i > 10:
        break
    print(i)


1
1
2
3
5
8


There is no way to implement this behavior with a `list` comprehension. The following results in an infinite loop!

In [7]:
# we cant break list comprehension

[i for i in fibonacci() if i <= 10]

KeyboardInterrupt: 

# Dict comprehensions: Basic syntax

- Basic syntax: `{key: value for key, value in iterable}`
- The curly braces are necessary!

Let's first consider how you could create a `dict` that maps animal species onto their class. Ideally, we would like to Capitalize the strings!

In [14]:
SPECIES = 'whale', 'grasshopper', 'lizard'
CLASS = 'mammal', 'insect', 'reptile'

d = dict(zip(SPECIES, CLASS))
print(d)

{'whale': 'mammal', 'grasshopper': 'insect', 'lizard': 'reptile'}


In [15]:
d = {}
for species, class_ in zip(SPECIES, CLASS):
    d[species.capitalize()] = class_.capitalize()
print(d)

{'Whale': 'Mammal', 'Grasshopper': 'Insect', 'Lizard': 'Reptile'}


In [16]:
d = {species.capitalize(): class_.capitalize() for species, class_ in zip(SPECIES, CLASS)}
print(d)

{'Whale': 'Mammal', 'Grasshopper': 'Insect', 'Lizard': 'Reptile'}


#### Filtering dict comprehensions

- Filtering syntax: `{key: value for key, value in iterable if expression}`
- This is an alternative (in many cases) to the continue statement

Let's say that we don't want to include insects! Without a `dict` comprehension, we might do this as follows:

In [17]:
d = {}
for species, class_ in zip(SPECIES, CLASS):
    if class_ == 'insect':
        continue
    d[species.capitalize()] = class_.capitalize()
print(d)

{'Whale': 'Mammal', 'Lizard': 'Reptile'}


With a `dict` comprehension, this becomes:

In [18]:
d = {
    species.capitalize(): class_.capitalize()
    for species, class_ in zip(SPECIES, CLASS)
    if class_ != 'insect'
}
print(d)

{'Whale': 'Mammal', 'Lizard': 'Reptile'}


# Generator expressions: Basic syntax

- Basic syntax: `(expression for element in iterable)`
- Parentheses are optional if leaving them does not result in ambiguity
- Because they are generators, they are *lazy* and not evaluated immediately. You need actively iterate through them!

In [19]:
from math import sqrt

# create generator
g = (sqrt(i) for i in range(5))
for i in g:
    print(i)

0.0
1.0
1.4142135623730951
1.7320508075688772
2.0


In [25]:
# g is generator
print(g)

<generator object <genexpr> at 0x000001F5601C7370>


Because they are generators, you can iterate over them using `next(g)` until you get a stopiteration.

In [21]:
g = (sqrt(i) for i in range(5))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

0.0
1.0
1.4142135623730951
1.7320508075688772
2.0


StopIteration: 

## Filtering generator expressions

- Filtering syntax: `(expression for element in iterable if expression)`
- This is an alternative (in many cases) to the `continue` statement

Let's consider a `for` loop that skips all odd numbers:

In [22]:
g = (i for i in range(10) if not i%2)
for i in g:
    print(i)

0
2
4
6
8


#### Breaking generator expressions (deprecated)

It is, in principle, possible to break a generator expression by explicitly raising a `StopIteration`. However, this behavior has been deprecated and as of Python 3.6 gives a `DeprecationWarning`.

In [28]:
def fibonacci():
    
    yield 1
    yield 1
    l = [1, 1]
    while True:
        l = [l[-1], sum(l[-2:])]
        yield l[-1]

        
def stop():
    raise StopIteration()
    
    
g = (i for i in fibonacci() if i < 10 or stop())
for i in g:
    print(i)

# avoid stopiteration error by implement wrap()

1
1
2
3
5
8


RuntimeError: generator raised StopIteration

Usually, you can avoid this construction by rewriting code. And if you *really want*, you can emulate by passing a custom `Exception` and catching this in a custom `wrap()` function.

In [36]:
class EndGenerator(Exception): pass

def stop():
    raise EndGenerator()
    
def wrap(g):
    
    l = []
    while True:
        try:
            l.append(next(g))
        except EndGenerator:
            break
    return l
    
    
g = wrap(i for i in fibonacci() if i < 10 or stop())
for i in g:
    print(i)


1
1
2
3
5
8


# Nested comprehensions

- Basic syntax: `[expression for element1 in iterable1 for element2 in iterable2]`

Let's say that you want to define a 2×2 grid of x,y coordinates. With a nested `for` loop, you could this as follows:

In [38]:
grid = []
for x in range(2):
    for y in range(2):
        grid.append((x,y))
print(grid)

[(0, 0), (0, 1), (1, 0), (1, 1)]


With a nested list comprehension, this becomes:

In [39]:
def g(s):
    print(s)
    yield 1
    print(s)
    yield 2

grid = [ (x, y) for x in g('x') for y in g('y') ]
print(grid)

x
y
y
x
y
y
[(1, 1), (1, 2), (2, 1), (2, 2)]


In [40]:
grid = [ (x, y) for x in range(2) for y in range(2) ]
print(grid)

[(0, 0), (0, 1), (1, 0), (1, 1)]
